In [1]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers, datasets, Sequential
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.random.set_seed(2345)

In [7]:
# 5 units of conv + max pooling
# 卷积层
conv_layers = [
    # unit 1
    layers.Conv2D(64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding="same"), 

    # unit 2
    layers.Conv2D(128, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(128, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding="same"), 

    # unit 3
    layers.Conv2D(256, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(256, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding="same"), 

    # unit 4
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding="same"), 

    # unit 5
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding="same"), 
]

def preprocess(x, y):
    # [0 ~ 1]
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x, y

(x, y), (x_test, y_test) = datasets.cifar100.load_data()
print(x.shape, y.shape, x_test.shape, y_test.shape)

train_db = tf.data.Dataset.from_tensor_slices((x, y))
train_db = train_db.shuffle(10000).map(preprocess).batch(64)

test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_db = test_db.map(preprocess).batch(64)

sample = next(iter(train_db))
print('sample:', sample[0].shape, sample[1].shape, tf.reduce_min(sample[0]), tf.reduce_max(sample[0]))

optimizer = optimizers.Adam(lr=1e-4)

# [b, 32, 32, 3] => [b, 1, 1, 512]
conv_net = Sequential(conv_layers)

# 全链接层
fc_net = Sequential([
    layers.Dense(256, activation=tf.nn.relu),
    layers.Dense(128, activation=tf.nn.relu),
    layers.Dense(100, activation=None)
])

conv_net.build(input_shape=[None, 32, 32, 3])
fc_net.build(input_shape=[None, 512])

# [1, 2] + [3, 4] => [1, 2, 3, 4] 
variables = conv_net.trainable_variables + fc_net.trainable_variables

for epoch in range(50):
    for step, (x, y) in enumerate(train_db):
        with tf.GradientTape() as tape:
            # [b, 32, 32, 3] => [b, 1, 1, 512]
            out = conv_net(x)
            # flatten, => [b, 512]
            out = tf.reshape(out, [-1, 512])
            # [b, 512] => [b, 100]
            logits = fc_net(out)
            # [b] => [b, 100]
            y_onehot = tf.one_hot(y, depth=100)
            # compute loss
            loss = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)
            loss = tf.reduce_mean(loss)
        
        grads = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(grads, variables))

        if step % 100 == 0:
            print(epoch, step, 'loss:', float(loss))

    total_num = 0
    total_correct = 0
    for x, y in test_db:
        out = conv_net(x)
        out = tf.reshape(out, [-1, 512])
        logits = fc_net(out)
        prob = tf.nn.softmax(logits, axis=1)
        pred = tf.argmax(prob, axis=1)
        pred = tf.cast(pred, dtype=tf.int32)

        correct = tf.cast(tf.equal(correct, y), dtype=tf.int32)
        correct = tf.reduce_sum(correct)

        total_num += x.shape[0]
        total_correct += int(correct)

    acc = total_correct / total_num
    print(epoch, 'acc:', acc)


(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
sample: (16, 32, 32, 3) (16, 1) tf.Tensor(0.0, shape=(), dtype=float32) tf.Tensor(1.0, shape=(), dtype=float32)
0 0 loss: 4.604842662811279
0 100 loss: 4.600252151489258
0 200 loss: 4.606687545776367
0 300 loss: 4.606091022491455
0 400 loss: 4.60454797744751
0 500 loss: 4.550275802612305
0 600 loss: 4.569247722625732
0 700 loss: 4.311275959014893
0 800 loss: 4.601819038391113
0 900 loss: 4.5080790519714355
0 1000 loss: 4.404435634613037
0 1100 loss: 4.071237564086914
0 1200 loss: 4.418947219848633
0 1300 loss: 4.329734802246094
0 1400 loss: 4.244333267211914
0 1500 loss: 4.130109786987305
0 1600 loss: 3.867675304412842
0 1700 loss: 4.537472724914551
0 1800 loss: 4.252060890197754
0 1900 loss: 4.3944597244262695
0 2000 loss: 4.253200531005859
0 2100 loss: 3.8479163646698
0 2200 loss: 4.462834358215332
0 2300 loss: 4.104136943817139
0 2400 loss: 4.178561210632324
0 2500 loss: 3.9681029319763184
0 2600 loss: 3.9120941162109375
0 

NameError: name 'correct' is not defined